<a href="https://colab.research.google.com/github/zhaoxiaoer/ml/blob/main/src/2020/hw4/hw4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recurrent Neural Network

本次作业是要让同学接触 NLP 当中一个简单的 task -- 语句分类（文本分类）

给定一个语句，判断他有没有恶意（负面标 1，正面标 0）

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Colab\ Notebooks/2020/hw4
!ls
path_prefix = './'

/content/drive/MyDrive/Colab Notebooks/2020/hw4
data.zip   testing_data.txt    training_nolabel.txt
hw4.ipynb  training_label.txt  w2v_all.model


### Download Dataset

有三个档案，分别是 training_label.txt、training_nolabel.txt、testing_data.txt

- training_label.txt：有 label 的 training data (句子配上 0 or 1, +++$+++ 只是分隔符号，不要理它)
      - e.g., 1 +++$+++ are wtf ... awww thanks !

- training_nolabel.txt：没有 label 的 training data (只有句子)，用来做 semi-supervised learning
      - ex: hates being this burnt !! ouch

- testing_data.txt：你要判断 testing data 里面的句子是 0 or 1

    >id,text

    >0,my dog ate our dinner . no, seriously ... he ate it .

    >1,omg last day sooon n of primary noooooo x im gona be swimming out of school wif the amount of tears am gona cry
    
    >2,stupid boys .. they ' re so .. stupid !

In [ ]:
!gdown --id '1lz0Wtwxsh5YCPdqQ3E3l_nbfJT1N13V8' --output data.zip
!unzip data.zip
!ls

Downloading...
From: https://drive.google.com/uc?id=1lz0Wtwxsh5YCPdqQ3E3l_nbfJT1N13V8
To: /content/drive/My Drive/Colab Notebooks/2020/hw4/data.zip
45.1MB [00:00, 107MB/s] 
Archive:  data.zip
  inflating: training_label.txt      
  inflating: testing_data.txt        
  inflating: training_nolabel.txt    
data.zip  hw4.ipynb  testing_data.txt  training_label.txt  training_nolabel.txt


In [ ]:
with open('testing_data.txt', 'r') as f:
    line = f.readline()
    line = f.readline()
    print(line)
    # print(line.strip('\n').split(' '))
    # print(line.strip('\n').split(' ')[2:])
    print(line.strip('\n').split(','))
    print(line.strip('\n').split(',')[1:])
    print("".join(line.strip('\n').split(',')[1:]))
    print("".join(line.strip('\n').split(',')[1:]).strip())

0,my dog ate our dinner . no , seriously ... he ate it .

['0', 'my dog ate our dinner . no ', ' seriously ... he ate it .']
['my dog ate our dinner . no ', ' seriously ... he ate it .']
my dog ate our dinner . no  seriously ... he ate it .
my dog ate our dinner . no  seriously ... he ate it .


In [15]:
# this is for filtering the warnings
import warnings
warnings.filterwarnings('ignore')

### utils

In [3]:
# utils.py
# 这个 block 用来定义一些常用的函数
import torch
import numpy as np
import pandas as pd
import torch.optim as optim
import torch.nn.functional as F

def load_training_data(path='training_label.txt'):
    # 把 training 时需要的 data 读进来
    # 如果是 'training_label.txt'，需要读取 label，如果是 'training_nolabel.txt'，不需要读取 label
    if 'training_label' in path:
        with open(path, 'r') as f:
            lines = f.readlines()
            lines = [line.strip('\n').split(' ') for line in lines]
        x = [line[2:] for line in lines]
        y = [line[0] for line in lines]
        return x, y
    else:
        with open(path, 'r') as f:
            lines = f.readlines()
            x = [line.strip('\n').split(' ') for line in lines]
        return x

def load_testing_data(path='testing_data.txt'):
    # 把 testing 时需要的 data 读进来
    with open(path, 'r') as f:
        lines = f.readlines()
        X = ["".join(line.strip('\n').split(',')[1:]).strip() for line in lines[1:]]
        X = [sen.split(' ') for sen in X]
    return X

def evaluation(outputs, labels):
    # outputs => probability (float)
    # labels => labels
    outputs[outputs>=0.5] = 1  # 大于等于 0.5 为正面
    outputs[outputs<0.5] = 0  # 小于 0.5 为负面
    correct = torch.sum(torch.eq(outputs, labels)).item()
    return correct

In [6]:
outputs = torch.tensor([[0.8], [0.3], [0.6]])
print(outputs)
print(outputs>=0.5)
print(outputs[outputs>=0.5])
outputs[outputs>=0.5] = 1
print(outputs)
print(outputs.size())

tensor([[0.8000],
        [0.3000],
        [0.6000]])
tensor([[ True],
        [False],
        [ True]])
tensor([0.8000, 0.6000])
tensor([[1.0000],
        [0.3000],
        [1.0000]])
torch.Size([3, 1])


### Train Word to Vector

In [4]:
# w2v.py
# 这个 block 是用来训练 word to vector 的 word embedding
# 注意！这个 block 在训练 word to vector 时是用 cpu，可能要花费 10 分钟以上
import os
import numpy as np
import pandas as pd
import argparse
from gensim.models import word2vec

def train_word2vec(x):
    # 训练 word to vector 的 word embedding
    # sentences: 可以是一个 list，对于大语料集，建议使用 BrownCorpus, Text8Corpus 或 lineSentence 构建
    # size: 特征向量的维度，默认100
    # window: 窗口大小，表示当前词与预测词在一个句子中的最大距离是多少
    # min_count: 可以对字典做截断，词频少于 min_count 次数的单词会被丢掉，默认值5
    # max_vocab_size: 设置词向量构建期间的 RAM 限制，设置成 None 则没有限制
    # workers: 并发训练时的线程数
    # sg: 用于设置训练算法，默认是0，对应 CBOW 算法，1 对应 skip-gram 算法
    # iter: 迭代次数，默认是5
    model = word2vec.Word2Vec(x, size=250, window=5, min_count=5, workers=12, iter=10, sg=1)
    return model

if __name__ == "__main__":
    print("loading training data ...")
    train_x, y = load_training_data('training_label.txt')
    print(len(train_x))
    print(type(train_x))
    print(train_x[0])
    print(y[0])
    train_x_no_label = load_training_data('training_nolabel.txt')

    print("loading testing data ...")
    test_x = load_testing_data('testing_data.txt')

    # model = train_word2vec(train_x + train_x_no_label + test_x)
    model = train_word2vec(train_x + test_x)

    print("saving model ...")
    model.save(os.path.join(path_prefix, 'w2v_all.model'))  # 模型保存

loading training data ...
200000
<class 'list'>
['are', 'wtf', '...', 'awww', 'thanks', '!']
1
loading testing data ...
saving model ...


In [5]:
from gensim.models import Word2Vec
model2 = Word2Vec.load('w2v_all.model')  # 模型读取
print(model2.vector_size)
print(type(model2['dog']))
print(len(model2['dog']))
# print(model2['dog'])
print(model2.similarity('dog', 'cat'))  # 相似性
print(model2.most_similar(positive=['woman', 'king'], negative=['man'], topn=1))  # 最相似
print(model2.most_similar(['man']))
print(model2.doesnt_match("breakfast cereal dinner lunch".split()))  # 最不相似
print(type(model2.wv.vocab))
print(len(model2.wv.vocab))

250
<class 'numpy.ndarray'>
250
0.57638174
[('devil', 0.3734397292137146)]
[('dude', 0.5109483003616333), ('lawd', 0.4925040304660797), ('mang', 0.4771396219730377), ('maaan', 0.47376304864883423), ('guy', 0.45980724692344666), ('wellz', 0.44957679510116577), ('flyy', 0.4483198821544647), ('raul', 0.44600674510002136), ('jeez', 0.44435152411460876), ('babe', 0.4417971670627594)]
cereal
<class 'dict'>
24694


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
 

### Data Preprocess

In [6]:
# preprocess.py
# 这个 block 用来做 data 的预处理
from torch import nn
from gensim.models import Word2Vec

class Preprocess():
    def __init__(self, sentences, sen_len, w2v_path="./w2v_all.model"):
        self.w2v_path = w2v_path
        self.sentences = sentences
        self.sen_len = sen_len
        self.idx2word = []
        self.word2idx = {}
        self.embedding_matrix = []
    def get_w2v_model(self):
        # 把之前训练好的 word to vec 模型读进来
        self.embedding = Word2Vec.load(self.w2v_path)
        self.embedding_dim = self.embedding.vector_size
    def add_embedding(self, word):  # 添加两个额外的单词，用于"填充"和"未定义"
        # 把 word 加进 embedding，并赋予它一个随机生成的 representation vector
        # word 只会是 "<PAD>" 或 "<UNK>"
        vector = torch.empty(1, self.embedding_dim)
        torch.nn.init.uniform_(vector)
        self.word2idx[word] = len(self.word2idx)
        self.idx2word.append(word)
        self.embedding_matrix = torch.cat([self.embedding_matrix, vector], 0)
    def make_embedding(self, load=True):
        print("Get embedding ...")
        # 获取训练好的 Word2Vec word embedding
        if load:
            print("loading word to vec model ...")
            self.get_w2v_model()
        else:
            raise NotImplementedError
        # 制作一个 word2idx 的 dictionary
        # 制作一个 idx2word 的 list
        # 制作一个 word2vector 的 list
        for i, word in enumerate(self.embedding.wv.vocab):  # 模型的词向量
            print('get words #{}'.format(i+1), end='\r')
            # e.g. self.word2idx['he'] = 1
            # e.g. self.idx2word[1] = 'he'
            # e.g. slef.vectors[1] = 'he' vector
            self.word2idx[word] = len(self.word2idx)
            self.idx2word.append(word)
            self.embedding_matrix.append(self.embedding[word])
        print('')
        self.embedding_matrix = torch.tensor(self.embedding_matrix)
        # 将 "<PAD>" 和 "<UNK>" 加入 embedding 里面
        self.add_embedding("<PAD>")
        self.add_embedding("<UNK>")
        print("total words: {}".format(len(self.embedding_matrix)))
        return self.embedding_matrix
    def pad_sequence(self, sentence):
        # 将每个句子变成一样的长度
        if len(sentence) > self.sen_len:
            sentence = sentence[:self.sen_len]
        else:
            pad_len = self.sen_len - len(sentence)
            for _ in range(pad_len):
                sentence.append(self.word2idx["<PAD>"])
        assert len(sentence) == self.sen_len
        return sentence
    def sentence_word2idx(self):
        sentence_list = []
        for i, sen in enumerate(self.sentences):
            print('sentence count #{}'.format(i+1), end='\r')
            sentence_idx = []
            for word in sen:
                if (word in self.word2idx.keys()):
                    sentence_idx.append(self.word2idx[word])
                else:
                    sentence_idx.append(self.word2idx["<UNK>"])
            # 将每个句子变成一样的长度
            sentence_idx = self.pad_sequence(sentence_idx)
            sentence_list.append(sentence_idx)
        return torch.LongTensor(sentence_list)
    def labels_to_tensor(self, y):
        # 把 labels 转成 tensor
        y = [int(label) for label in y]
        return torch.LongTensor(y)

In [7]:
vector = torch.empty(1, 3)
print(vector)
torch.nn.init.uniform_(vector)
print(vector)

tensor([[7.9301e+13, 3.0702e-41, 3.3631e-44]])
tensor([[0.5401, 0.2763, 0.4586]])


### Dataset

In [8]:
# data.py
# 实现了 dataset 所需要的 '__init__', '__getitem__', '__len__'
# 好让 dataloader 能使用
import torch
from torch.utils import data

class TwitterDataset(data.Dataset):
    """
    Expected data shape like:(data_num, data_len)
    Data can be a list of numpy array or a list of lists
    input data shape:(data_num, seq_len, feature_dim)

    __len__ will return the number of data
    """
    def __init__(self, X, y):
        self.data = X
        self.label = y
    def __getitem__(self, idx):
        if self.label is None: return self.data[idx]
        return self.data[idx], self.label[idx]
    def __len__(self):
        return len(self.data)

### Model

In [9]:
# model.py
# 这个 block 是训练的模型
import torch
from torch import nn
class LSTM_Net(nn.Module):
    def __init__(self, embedding, hidden_dim, num_layers, dropout=0.5, fix_embedding=True):
        super(LSTM_Net, self).__init__()
        # 制作 embedding layer
        self.embedding = torch.nn.Embedding(embedding.size(0), embedding.size(1))
        self.embedding.weight = torch.nn.Parameter(embedding)
        # 是否将 embedding fix 住，如果 fix_embedding 为 False，在训练过程中，embedding 也会跟着被训练
        self.embedding.weight.requires_grad = False if fix_embedding else True
        self.embedding_dim = embedding.size(1)
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.dropout = dropout
        self.lstm = nn.LSTM(self.embedding_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.classifier = nn.Sequential( nn.Dropout(dropout),
                                         nn.Linear(hidden_dim, 1),
                                         nn.Sigmoid() )
    def forward(self, inputs):
        inputs = self.embedding(inputs)
        x, _ = self.lstm(inputs, None)
        # x 的 dimension (batch, seq_len, hidden_dim)
        # 取用 LSTM 最后一层的 hidden state （hiden state 等于 输出）
        x = x[:, -1, :]
        x = self.classifier(x)
        return x

### Train

In [13]:
# train.py
# 这个 block 是用来训练模型的
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F

def training(batch_size, n_epoch, lr, model_dir, train, valid, model, device):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print('\nstart training, parameter total:{}, trainable:{}\n'.format(total, trainable))
    model.train()  # 将 model 的模式设为 train，这样 optimizer 就可以更新 model 的参数
    criterion = nn.BCELoss()  # 定义损失函数，这里我们使用 binary cross entropy loss
    t_batch = len(train)
    v_batch = len(valid)
    optimizer = optim.Adam(model.parameters(), lr=lr)  # 将模型的参数给 optimizer，并给予适当的 learning rate
    total_loss, total_acc, best_acc = 0, 0, 0
    for epoch in range(n_epoch):
        total_loss, total_acc = 0, 0
        # 这段做 training
        for i, (inputs, labels) in enumerate(train):
            inputs = inputs.to(device, dtype=torch.long)  # device 为 "cuda"，将 inputs 转成 torch.cuda.LongTensor
            labels = labels.to(device, dtype=torch.float)  # device 为 "cuda"，将 labels 转成 torch.cuda.FloatTensor，因为等会要喂入 criterion，所以类型须是 float
            optimizer.zero_grad()  # 由于 loss.backward() 的 gradient 会累加，所以每次喂完一个 batch 后需要清零
            outputs = model(inputs)  # 将 input 喂给模型
            outputs = outputs.squeeze()  # 去掉维度为1的 dimension，好让 outputs 可以喂入 criterion()
            loss = criterion(outputs, labels)  # 计算此时模型的 training loss
            loss.backward()  # 算 loss 的 gradient
            optimizer.step()  # 更新训练模型的参数
            correct = evaluation(outputs, labels)
            total_acc += (correct / batch_size)
            total_loss += loss.item()
            print('[ Epoch{}: {}/{} ] loss:{:.3f} acc:{:.3f} '.format(
                epoch+1, i+1, t_batch, loss.item(), correct*100/batch_size))
        print('\nTrain | Loss: {:.5f} Acc: {:.3f}'.format(total_loss/t_batch, total_acc/t_batch*100))

        # 这段做 validation
        model.eval()  # 将 model 的模式设为 eval，这样 model 的参数就会被固定住
        with torch.no_grad():
            total_loss, total_acc = 0, 0
            for i, (inputs, labels) in enumerate(valid):
                inputs = inputs.to(device, dtype=torch.long)
                labels = labels.to(device, dtype=torch.float)
                outputs = model(inputs)
                outputs = outputs.squeeze()
                loss = criterion(outputs, labels)
                correct = evaluation(outputs, labels)
                total_acc += (correct / batch_size)
                total_loss += loss.item()
            print("Valid | Loss: {:.5f} Acc: {:.3f} ".format(total_loss/v_batch, total_acc/v_batch*100))
            if total_acc > best_acc:
                # 如果 validation 的结果优于之前的所有结果，就把当下的模型保存下来，用于预测使用
                best_acc = total_acc
                #torch.save(model, "{}/val_acc_{:.3f}.model".format(model_dir,total_acc/v_batch*100))
                torch.save(model, "{}/ckpt.model".format(model_dir))
                print('saving model with acc {:.3f}'.format(total_acc/v_batch*100))
        print('------------------------------------')
        model.train()  # 将 model 的模式设为 train，这样 optimizer 就可以更新 model 的参数 (因为刚刚设成 eval 模式)

### Test

In [11]:
# test.py
# 这个 block 用来对 testing_data.txt 做预测
import torch
from torch import nn

def testing(test_loader, model, device):
    model.eval()
    ret_output = []
    with torch.no_grad():
        for i, inputs in enumerate(test_loader):
            inputs = inputs.to(device, dtype=torch.long)
            outputs = model(inputs)
            outputs = outputs.squeeze()
            outputs[outputs>=0.5] = 1  # 大于等于 0.5 为正面
            outputs[outputs<0.5] = 0  # 小于 0.5 为负面
            ret_output += outputs.int().tolist()
    
    return ret_output

### Main

In [16]:
# main.py
import os
import torch

# 通过 torch.cuda.is_available() 的返回值判断是否有 GPU 的环境，如果有的话 device 就设为 "cuda"，没有的话就设为 "cpu"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 处理好各个 data 的路径
train_with_label = os.path.join(path_prefix, 'training_label.txt')
train_no_label = os.path.join(path_prefix, 'training_nolabel.txt')
testing_data = os.path.join(path_prefix, 'testing_data.txt')

w2v_path = os.path.join(path_prefix, 'w2v_all.model')  # 处理 word to vec model 的路径

# 定义：句子长度、是否固定 embedding、batch 大小、训练几个 epoch、learning rate 的值、model 的文件夹路径
sen_len = 20
fix_embedding = True  # fix embedding during training
batch_size = 128
epoch = 50
lr = 0.001
model_dir = path_prefix

print("loading data ...")  # 把 'training_label.txt' 跟 'training_nolabel.txt' 读进来
train_x, y = load_training_data(train_with_label)
train_x_no_label = load_training_data(train_no_label)

# 对 inputs 和 labels 做预处理
preprocess = Preprocess(train_x, sen_len, w2v_path=w2v_path)
embedding = preprocess.make_embedding(load=True)
train_x = preprocess.sentence_word2idx()
y = preprocess.labels_to_tensor(y)

# 制作一个 model 对象
model = LSTM_Net(embedding, hidden_dim=150, num_layers=1, dropout=0.5, fix_embedding=fix_embedding)
model = model.to(device)

# 把 data 分为 training data 和 validation data (将一部分 training data 拿来当做 validation data)
X_train, X_val, y_train, y_val = train_x[:180000], train_x[180000:], y[:180000], y[180000:]

# 把 data 做成 dataset 供 dataloader 使用
train_dataset = TwitterDataset(X=X_train, y=y_train)
val_dataset = TwitterDataset(X=X_val, y=y_val)

# 把 data 转成 batch of tensors
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           num_workers=8)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                         batch_size=batch_size,
                                         shuffle=False,
                                         num_workers=8)

# 开始训练
training(batch_size, epoch, lr, model_dir, train_loader, val_loader, model, device)

loading data ...
Get embedding ...
loading word to vec model ...
get words #24694
total words: 24696

start training, parameter total:6415351, trainable:241351


Train | Loss: 0.50090 Acc: 74.716
Valid | Loss: 0.45716 Acc: 77.772 
saving model with acc 77.772
------------------------------------

Train | Loss: 0.44328 Acc: 79.117
Valid | Loss: 0.44154 Acc: 79.051 
saving model with acc 79.051
------------------------------------

Train | Loss: 0.42744 Acc: 80.098
Valid | Loss: 0.42908 Acc: 79.722 
saving model with acc 79.722
------------------------------------

Train | Loss: 0.41467 Acc: 80.806
Valid | Loss: 0.42589 Acc: 80.011 
saving model with acc 80.011
------------------------------------

Train | Loss: 0.40243 Acc: 81.388
Valid | Loss: 0.42328 Acc: 80.140 
saving model with acc 80.140
------------------------------------

Train | Loss: 0.39062 Acc: 82.099
Valid | Loss: 0.43337 Acc: 79.593 
------------------------------------

Train | Loss: 0.37857 Acc: 82.701
Valid | Loss: 0.4

### Predict and Write to csv file

In [17]:
# 开始测试模型并进行预测
print("loading testing data ...")
test_x = load_testing_data(testing_data)
preprocess = Preprocess(test_x, sen_len, w2v_path=w2v_path)
embedding = preprocess.make_embedding(load=True)
test_x = preprocess.sentence_word2idx()
test_dataset = TwitterDataset(X=test_x, y=None)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          num_workers=8)
print('\nload model ...')
model_new = torch.load(os.path.join(model_dir, 'ckpt.model'))
outputs = testing(test_loader, model, device)

# 写入 csv 档案
tmp = pd.DataFrame({"id": [str(i) for i in range(len(test_x))], "labels": outputs})
print("save csv ...")
tmp.to_csv(os.path.join(path_prefix, 'predict.csv'), index=False)
print("Finishing Predicting")

loading testing data ...
Get embedding ...
loading word to vec model ...
get words #24694
total words: 24696

load model ...
save csv ...
Finishing Predicting


In [18]:
!pwd
!ls

/content/drive/My Drive/Colab Notebooks/2020/hw4
ckpt.model  hw4.ipynb	 testing_data.txt    training_nolabel.txt
data.zip    predict.csv  training_label.txt  w2v_all.model
